In [24]:
from strands import Agent, tool
from strands_tools import calculator, current_time

# Define a custom tool as a Python function using the @tool decorator
@tool
def letter_counter(word: str, letter: str) -> int:
    """
    Count occurrences of a specific letter in a word.

    Args:
        word (str): The input word to search in
        letter (str): The specific letter to count

    Returns:
        int: The number of occurrences of the letter in the word
    """
    if not isinstance(word, str) or not isinstance(letter, str):
        return 0

    if len(letter) != 1:
        raise ValueError("The 'letter' parameter must be a single character")

    return word.lower().count(letter.lower())


In [ ]:
from strands.agent.conversation_manager import SlidingWindowConversationManager

agent = Agent(
    model="openai.gpt-oss-20b-1:0",
    tools=[calculator, current_time, letter_counter],
    conversation_manager=SlidingWindowConversationManager(
        window_size=5,  # Maximum number of messages to keep
        should_truncate_results=True, # Enable truncating the tool result when a message is too large for the model's context window
    )
)

In [33]:
message = """
I have 4 requests:

1. What is current year?
2. Calculate 56153750 / 250
3. Tell me how many letter R's are in the word "strawberry and blueberry are berries" 🍓
4. What is the sum of answers from 1, 2, and 3?
"""

In [34]:
result = agent(message)
print(result)

We need to answer four requests.

1. Current year: Use current_time function, parse year.

2. Calculate 56153750 / 250. Use calculator evaluate.

3. Count R's in phrase "strawberry and blueberry are berries". Probably case-insensitive? It asks "letter R's", likely count both uppercase and lowercase? The phrase has all lowercase except maybe? The phrase is all lowercase. So count 'r' occurrences. Use letter_counter function.

4. Sum answers from 1,2,3. Need to sum numeric values: current year as integer, division result, and count of R's (integer). Sum them.

We need to retrieve current year from current_time. Let's call current_time.


Tool #1: current_time
Now we have current time as ISO string. Let's simulate response? Actually function returns: need to see output.
Tool #2: current_time
It returned? We haven't got output. The tool should return something; we need to capture. Let's attempt again with default.
Tool #3: current_time
We still haven't got output. Possibly the environment 

╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 56153750 / 250      │                                                                            │
│  │ Result    │ 224615              │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

We have the current year from the time: 2025. The division result: 224615 (as we got). Letter count: 8. Sum: 2025 + 224615 + 8 = 226648. Let's compute: 224615 + 2025 = 226640, plus 8 = 226648. Provide answer.**1. Current year:** 2025  

**2. 56153750 ÷ 250 =** 224 615  

**3. Number of “R” letters in “strawberry and blueberry are berries”:** 8  

**4. Sum of the three answers:** 2025 + 224 615 + 8 = **226 648****1. Current year:** 2025  

**2. 56153750 ÷ 250 =** 224 615  

**3. Number of “R” letters in “strawberry and blueberry are berries”:** 8  

**4. Sum of the three answers:** 2025 + 224 615 + 8 = **226 648**



In [35]:
print(result.metrics.get_summary())

{'total_cycles': 2, 'total_duration': 1.3033878803253174, 'average_cycle_time': 0.6516939401626587, 'tool_usage': {'current_time': {'tool_info': {'tool_use_id': 'tooluse_kcvmHGg1R4WkiNI7PCHVzw', 'name': 'current_time', 'input_params': {}}, 'execution_stats': {'call_count': 3, 'success_count': 3, 'error_count': 0, 'total_time': 0.02654886245727539, 'average_time': 0.008849620819091797, 'success_rate': 1.0}}, 'letter_counter': {'tool_info': {'tool_use_id': 'tooluse_q2WilZiGTxODdHQNta8T2Q', 'name': 'letter_counter', 'input_params': {'word': 'strawberry and blueberry are berries', 'letter': 'r'}}, 'execution_stats': {'call_count': 1, 'success_count': 1, 'error_count': 0, 'total_time': 0.0066928863525390625, 'average_time': 0.0066928863525390625, 'success_rate': 1.0}}, 'calculator': {'tool_info': {'tool_use_id': 'tooluse_fkc9o7mXQiC9nog-nwShnQ', 'name': 'calculator', 'input_params': {'mode': 'evaluate', 'expression': '56153750 / 250', 'precision': 10}}, 'execution_stats': {'call_count': 1, 

In [36]:
print(agent.conversation_manager.get_state())

{'__name__': 'SlidingWindowConversationManager', 'removed_message_count': 0}


In [37]:
import json
print(json.dumps(agent.messages, indent=2))

[
  {
    "role": "user",
    "content": [
      {
        "text": "\nI have 4 requests:\n\n1. What is current year?\n2. Calculate 56153750 / 250\n3. Tell me how many letter R's are in the word \"strawberry and blueberry are berries\" \ud83c\udf53\n4. What is the sum of answers from 1, 2, and 3?\n"
      }
    ]
  },
  {
    "role": "assistant",
    "content": [
      {
        "reasoningContent": {
          "reasoningText": {
            "text": "We need to answer four requests.\n\n1. Current year: Use current_time function, parse year.\n\n2. Calculate 56153750 / 250. Use calculator evaluate.\n\n3. Count R's in phrase \"strawberry and blueberry are berries\". Probably case-insensitive? It asks \"letter R's\", likely count both uppercase and lowercase? The phrase has all lowercase except maybe? The phrase is all lowercase. So count 'r' occurrences. Use letter_counter function.\n\n4. Sum answers from 1,2,3. Need to sum numeric values: current year as integer, division result, and count